In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.optim import lr_scheduler
import os

import shutil

from PIL import Image
from torch.utils.data import Dataset, DataLoader
import pandas as pd

from tqdm import tqdm

In [ ]:
#!unzip -q /content/lnu-deep-learn-1-image-classification-2025.zip -d /content/fast_data

#!rm /content/lnu-deep-learn-1-image-classification-2025.zip

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Використовуємо пристрій: {device}")
if device.type == 'cuda':
    print(f"Назва відеокарти: {torch.cuda.get_device_name(0)}")

data_dir = 'fast_data'
train_dir = os.path.join(data_dir, 'train')

Використовуємо пристрій: cuda
Назва відеокарти: Tesla T4


In [ ]:
os.chdir('/content')

print("Поточна папка:", os.getcwd())
print("Вміст папки:", os.listdir())

Поточна папка: /content
Вміст папки: ['.config', 'drive', 'fast_data', 'sample_data']


In [ ]:
classes_path = os.path.join(data_dir, 'classes.txt')

with open(classes_path, 'r') as f:
    class_names = [line.strip() for line in f.readlines()]

num_classes = len(class_names)

print(f"Кількість класів: {num_classes}")
print(f"Перші 5 класів для прикладу: {class_names[:5]}")

Кількість класів: 100
Перші 5 класів для прикладу: ['ADONIS', 'AFRICAN GIANT SWALLOWTAIL', 'AMERICAN SNOOT', 'AN 88', 'APPOLLO']


In [ ]:
train_dir = '/content/fast_data/train'

data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(), 
    transforms.Resize((256, 256)),      
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms)

num_classes = len(train_dataset.classes)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,       
    shuffle=True,
    num_workers=2,       
    pin_memory=True      
)

print(f"✅ Дані завантажено з: {train_dir}")
print(f"Класів: {num_classes}")
print(f"Зображень для тренування: {len(train_dataset)}")

✅ Дані завантажено з: /content/fast_data/train
Класів: 100
Зображень для тренування: 10911


In [ ]:
import torch.nn as nn
from torchvision import models


print("Завантажуємо ConvNeXt Base...")
model = models.convnext_base(weights=models.ConvNeXt_Base_Weights.DEFAULT)

num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, num_classes)

model = model.to(device)

print(f"✅ ConvNeXt Base оновлено. Вхідний розмір фінального шару: {num_ftrs}.")
print("Новий фінальний шар:", model.classifier[-1])

Завантажуємо ConvNeXt Base...
Downloading: "https://download.pytorch.org/models/convnext_base-6075fbad.pth" to /root/.cache/torch/hub/checkpoints/convnext_base-6075fbad.pth


100%|██████████| 338M/338M [00:01<00:00, 190MB/s]


✅ ConvNeXt Base оновлено. Вхідний розмір фінального шару: 1024.
Новий фінальний шар: Linear(in_features=1024, out_features=100, bias=True)


In [8]:
print(model)

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=128, out_features=512, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=512, out_features=128, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(

In [ ]:
for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

for param in model.features[7].parameters():
    param.requires_grad = True

for param in model.features[6].parameters():
    param.requires_grad = True

In [ ]:
criterion = nn.CrossEntropyLoss()
best_acc = 0.0
patience = 5               
epochs_without_improvement = 0
checkpoint_path = '/content/drive/MyDrive/butterfly_project/convnext_pre_butterfly_model.pth'


EPOCHS_PHASE_1 = 10        
EPOCHS_PHASE_2 = 20         
TOTAL_EPOCHS = EPOCHS_PHASE_1 + EPOCHS_PHASE_2


optimizer_1 = optim.Adam(model.classifier.parameters(), lr=0.001)
scheduler_1 = lr_scheduler.StepLR(optimizer_1, step_size=3, gamma=0.5)


print("\n--- ФАЗА 1/2: НАВЧАННЯ ТІЛЬКИ НОВОГО КЛАСИФІКАТОРА (10 ЕПОХ) ---")


current_optimizer = optimizer_1
current_scheduler = scheduler_1

for epoch in range(EPOCHS_PHASE_1):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in tqdm(train_loader, desc=f"Фаза 1 Епоха {epoch+1}"):
        inputs, labels = inputs.to(device), labels.to(device)

        current_optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        current_optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_preds += labels.size(0)
        correct_preds += (predicted == labels).sum().item()

    current_scheduler.step()
    epoch_acc = 100 * correct_preds / total_preds
    epoch_loss = running_loss / len(train_loader)
    current_lr = current_optimizer.param_groups[0]['lr']

    print(f"\n🏁 ФАЗА 1, Епоха [{epoch+1}/{EPOCHS_PHASE_1}] | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}% | LR: {current_lr:.6f}")


    if epoch_acc > best_acc:
        best_acc = epoch_acc
        epochs_without_improvement = 0
        torch.save(model.state_dict(), checkpoint_path)
        print(f"✅ Рекорд! Модель збережено (Acc: {best_acc:.2f}%)")
    else:
        epochs_without_improvement += 1
        print(f"⏹️ Точність не покращилась. Очікування: {epochs_without_improvement}/{patience}")

    if epochs_without_improvement >= patience:
        print(f"🛑 Рання зупинка! Немає покращення {patience} епох.")
        break



if epochs_without_improvement < patience:

    print("\n--- ПЕРЕХІД ДО ФАЗИ 2/2: ТОНКЕ НАЛАШТУВАННЯ ВСІЄЇ МОДЕЛІ ---")
    current_optimizer = optim.Adam(model.parameters(), lr=0.0001) 
    current_scheduler = lr_scheduler.StepLR(current_optimizer, step_size=3, gamma=0.5)

    epochs_without_improvement = 0

    for epoch in range(EPOCHS_PHASE_2):
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in tqdm(train_loader, desc=f"Фаза 2 Епоха {epoch+1}"):
            inputs, labels = inputs.to(device), labels.to(device)

            current_optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            current_optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_preds += labels.size(0)
            correct_preds += (predicted == labels).sum().item()

        current_scheduler.step()
        epoch_acc = 100 * correct_preds / total_preds
        epoch_loss = running_loss / len(train_loader)
        current_lr = current_optimizer.param_groups[0]['lr']

        print(f"\n🔥 ФАЗА 2, Епоха [{epoch+1}/{EPOCHS_PHASE_2}] | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}% | LR: {current_lr:.6f}")


        if epoch_acc > best_acc:
            best_acc = epoch_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), checkpoint_path)
            print(f"✅ Рекорд! Модель збережено (Acc: {best_acc:.2f}%)")
        else:
            epochs_without_improvement += 1
            print(f"⏹️ Точність не покращилась. Очікування: {epochs_without_improvement}/{patience}")

        if epochs_without_improvement >= patience:
            print(f"🛑 Рання зупинка! Немає покращення {patience} епох.")
            break



print("\n🏆 ФІНАЛ. Тренування завершено!")
print(f"Найкраща точність (збережена): {best_acc:.2f}%")


--- ФАЗА 1/2: НАВЧАННЯ ТІЛЬКИ НОВОГО КЛАСИФІКАТОРА (10 ЕПОХ) ---


Фаза 1 Епоха 1: 100%|██████████| 171/171 [03:02<00:00,  1.07s/it]



🏁 ФАЗА 1, Епоха [1/10] | Loss: 2.5782 | Acc: 52.18% | LR: 0.001000
✅ Рекорд! Модель збережено (Acc: 52.18%)


Фаза 1 Епоха 2: 100%|██████████| 171/171 [02:57<00:00,  1.04s/it]



🏁 ФАЗА 1, Епоха [2/10] | Loss: 0.9961 | Acc: 83.01% | LR: 0.001000
✅ Рекорд! Модель збережено (Acc: 83.01%)


...


🔥 ФАЗА 2, Епоха [18/20] | Loss: 0.0472 | Acc: 98.70% | LR: 0.000002
⏹️ Точність не покращилась. Очікування: 2/5


Фаза 2 Епоха 19: 100%|██████████| 171/171 [02:58<00:00,  1.05s/it]



🔥 ФАЗА 2, Епоха [19/20] | Loss: 0.0506 | Acc: 98.49% | LR: 0.000002
⏹️ Точність не покращилась. Очікування: 3/5


Фаза 2 Епоха 20: 100%|██████████| 171/171 [02:59<00:00,  1.05s/it]


🔥 ФАЗА 2, Епоха [20/20] | Loss: 0.0488 | Acc: 98.67% | LR: 0.000002
⏹️ Точність не покращилась. Очікування: 4/5

🏆 ФІНАЛ. Тренування завершено!
Найкраща точність (збережена): 98.86%


In [ ]:
checkpoint_path = '/content/drive/MyDrive/butterfly_project/convnext_pre_butterfly_model.pth'

print("Створення порожньої архітектури ConvNeXt...")
model = models.convnext_base(weights=None)

num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, num_classes)


print("Завантажуємо ваги з чекпойнта...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.to(device)

model.eval()

print("🎉 Модель успішно завантажена і готова до передбачень (Inference)!")

Створення порожньої архітектури ConvNeXt...
Завантажуємо ваги з чекпойнта...
🎉 Модель успішно завантажена і готова до передбачень (Inference)!


In [ ]:
class ButterflyTestDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder_path, img_name)

        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, img_name


test_dir = '/content/fast_data/test'


test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


test_dataset = ButterflyTestDataset(test_dir, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False) 

print(f"Знайдено тестових зображень: {len(test_dataset)}")

Знайдено тестових зображень: 2728


In [ ]:
try:
    class_names = train_dataset.classes
except NameError:
    print("Відновлюємо список класів...")
    temp_data = datasets.ImageFolder('/content/fast_data/train')
    class_names = temp_data.classes

print(f"Всього класів: {len(class_names)}")

all_ids = []
all_preds = []

model.eval() 
model.to(device)

print("⏳ Починаю передбачення...")

with torch.no_grad(): 
    for inputs, filenames in tqdm(test_loader):
        inputs = inputs.to(device)

        outputs = model(inputs)

        _, predicted_indices = torch.max(outputs, 1)

        for i in range(len(filenames)):

            file_id = os.path.splitext(filenames[i])[0]
            all_ids.append(file_id)

            class_idx = predicted_indices[i].item()
            predicted_label = class_names[class_idx]
            all_preds.append(predicted_label)


submission_df = pd.DataFrame({
    'id': all_ids,
    'label': all_preds
})

csv_path = '/content/drive/MyDrive/butterfly_project/submission_convnext_pre.csv'
submission_df.to_csv(csv_path, index=False)

print(f"\n✅ Готово! Файл збережено: {csv_path}")
print("Ось перші 5 рядків для перевірки:")
print(submission_df.head())

Всього класів: 100
⏳ Починаю передбачення...


100%|██████████| 86/86 [00:37<00:00,  2.29it/s]


✅ Готово! Файл збережено: /content/drive/MyDrive/butterfly_project/submission_convnext_pre.csv
Ось перші 5 рядків для перевірки:
                                 id                    label
0  aea4ce0521db4db2bf6be0cb5e47126d  WHITE LINED SPHINX MOTH
1  8429d6c8faa5428bbdd5557fd7e47462            HERCULES MOTH
2  d7dac4265b6c4aff9f9b772e98ef6237                LUNA MOTH
3  33ebf85ba9f44f8686a1c412237ed47c     ARCIGERA FLOWER MOTH
4  2dbfe7eecb4c43afbdfccf1c9e83e9b6          IPHICLUS SISTER
